In [15]:
%%time
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import xgboost as xgb

from sklearn import preprocessing
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA
from math import sqrt
from scipy import stats
from scipy.stats import norm, skew #for some statistics

from rgf.sklearn import RGFRegressor,RGFClassifier

Wall time: 1 ms


In [4]:
# !pip3 install xgboost

  Using cached https://files.pythonhosted.org/packages/2e/bd/ad3a963b630fa3ee72d1a672fd207263fa0a18113688273afe8298293535/xgboost-0.82.tar.gz


Files/directories not found in C:\Users\GURVIN~1.SIN\AppData\Local\Temp\pip-install-4l5owkxx\xgboost\pip-egg-info
You are using pip version 19.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
from IPython.core.interactiveshell import InteractiveShell
from tqdm import tqdm_notebook
InteractiveShell.ast_node_interactivity = "all"

In [4]:
os.listdir('data_q2')

['data_test.csv', 'data_train.csv', 'README.txt', 'sample_submission.csv']

In [5]:
print(os.listdir("data_q2"))
train = pd.read_csv(r'data_q2/data_train.csv')
test = pd.read_csv(r'data_q2/data_test.csv')
sub = pd.read_csv(r'data_q2/sample_submission.csv')

['data_test.csv', 'data_train.csv', 'README.txt', 'sample_submission.csv']


In [6]:
train.head()
train.shape

,feature-1,feature-2,feature-3,feature-4,feature-5,feature-6,feature-7,feature-8,feature-9,feature-10,label
0,20.306201,7.246964,30.746074,70.738222,82.089549,44.537815,56.908667,17.261529,27.450978,42.425691,7
1,26.752616,16.396760,30.277686,39.271342,80.597014,31.302522,76.580799,50.305849,63.137250,29.514480,3
2,23.448827,14.291499,31.549011,57.424320,86.567170,37.394956,78.454333,44.758231,59.215678,34.578621,4
3,21.998386,10.850202,32.920712,65.819695,86.567170,39.075630,74.004684,50.591210,58.431376,38.060054,3
4,26.188554,10.364373,31.381732,63.537360,98.507465,36.554622,66.042155,52.787542,75.294116,40.018760,6


(6374, 11)

In [7]:
test.head()
test.shape

,feature-1,feature-2,feature-3,feature-4,feature-5,feature-6,feature-7,feature-8,feature-9,feature-10
0,0.000000,11.943319,100.000000,79.148787,0.000000,34.243699,66.276345,73.395226,71.764708,37.225929
1,21.917804,13.805665,34.024757,70.968638,37.313437,40.966388,79.625293,55.411921,78.431374,37.116899
2,22.965347,15.708501,30.511880,55.756012,86.567170,32.142856,77.283374,54.908921,64.705879,30.003979
3,25.946817,10.769232,30.645698,57.695894,98.507465,38.865545,66.510540,30.873128,65.882344,42.221736
4,23.045928,12.024289,29.909665,66.226727,86.567170,40.336134,76.580799,58.120087,59.607842,38.034107


(3000, 10)

In [8]:
# sub.head()

In [9]:
# Number of NaNs in each column
test.isnull().sum(axis=0)

feature-1     0
feature-2     0
feature-3     0
feature-4     0
feature-5     0
feature-6     0
feature-7     0
feature-8     0
feature-9     0
feature-10    0
dtype: int64

In [10]:
Y1=train['label']
train1=train.drop(['label'],axis=1)
train1=train1
Y=Y1.values

test1=test

In [11]:
#make 2 way interaction between all the columns
def Make_2way(X, Xt):
    columns_length=X.shape[1]
    for j in range (0,columns_length):
        for d in range (j+1,columns_length):  
            print(("Adding columns' interraction %d and %d" % (j, d) ))
            new_column_train=X[:,j]+X[:,d]
            new_column_test=Xt[:,j]+Xt[:,d]    
            X=np.column_stack((X,new_column_train))
            Xt=np.column_stack((Xt,new_column_test))
    return X, Xt

In [12]:
train2,test2= Make_2way(train1.values, test1.values)

Adding columns' interraction 0 and 1
Adding columns' interraction 0 and 2
Adding columns' interraction 0 and 3
Adding columns' interraction 0 and 4
Adding columns' interraction 0 and 5
Adding columns' interraction 0 and 6
Adding columns' interraction 0 and 7
Adding columns' interraction 0 and 8
Adding columns' interraction 0 and 9
Adding columns' interraction 1 and 2
Adding columns' interraction 1 and 3
Adding columns' interraction 1 and 4
Adding columns' interraction 1 and 5
Adding columns' interraction 1 and 6
Adding columns' interraction 1 and 7
Adding columns' interraction 1 and 8
Adding columns' interraction 1 and 9
Adding columns' interraction 2 and 3
Adding columns' interraction 2 and 4
Adding columns' interraction 2 and 5
Adding columns' interraction 2 and 6
Adding columns' interraction 2 and 7
Adding columns' interraction 2 and 8
Adding columns' interraction 2 and 9
Adding columns' interraction 3 and 4
Adding columns' interraction 3 and 5
Adding columns' interraction 3 and 6
A

In [47]:
##generate the cross validation fold.
# from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import r2_score
from sklearn.model_selection  import KFold,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
SEED=42
clf1 = lgb.LGBMClassifier()
# clf = xgb.XGBClassifier()
# {'learning_rate': 0.3264540354731975, 'max_bin': 100, 'max_depth': 40, 'min_child_samples': 46, 'min_child_weight': 1, 'n_estimators': 150, 'num_leaves': 10, 'reg_alpha': 4.632845512742032e-08, 'reg_lambda': 0.00103612872552685, 'scale_pos_weight': 0.2740310347625757, 'subsample_freq': 0}
# {'learning_rate': 0.4725970210346864, 'max_bin': 100, 'max_depth': 31, 'min_child_samples': 42, 'min_child_weight': 0, 'n_estimators': 150, 'num_leaves': 10, 'reg_alpha': 1e-09, 'reg_lambda': 8.389301713582564e-05, 'scale_pos_weight': 499.99999999999994, 'subsample_freq': 0}

clf = lgb.LGBMClassifier(max_bin = 100,max_depth=40,learning_rate=0.3264540,n_estimators=150,num_leaves=10,reg_alpha=4.63284,reg_lambda=0.001036,subsample=0.6143,scale_pos_weight=0.274031,subsample_freq=0,min_child_samples=46,min_child_weight=1)
st_train = train2
st_test = test2
clf2 = lgb.LGBMClassifier(max_bin = 100,max_depth=31,learning_rate=0.472597,n_estimators=150,num_leaves=10,reg_alpha=1e-09,reg_lambda=8.3893,subsample=0.6143,scale_pos_weight=499.92,subsample_freq=0,min_child_samples=46,min_child_weight=0)

clf = VotingClassifier(estimators = [('clf1', clf1),('clf2', clf2)],
                                       voting = 'soft', weights = [3,3])

# clf=CatBoostRegressor(iterations=80)
# clf = xgb.XGBRegressor(gamma = 0.76,learning_rate = 0.039,max_depth = 7,min_child_weight = 9.6754,n_estimators = 1100,subsample = 0.8,colsample_bytree = 0.99)
# clf = xgb.XGBRegressor(gamma = 0.7412,learning_rate = 0.0290,max_depth = 8,min_child_weight = 9.6754,n_estimators = 999) #96.00
# clf=ExtraTreesRegressor(n_estimators=10000, max_depth=9,  min_samples_leaf=1,  n_jobs=30, random_state=1)
# clf = RGFClassifier(max_leaf=500,algorithm="RGF",test_interval=100, loss="LS")

fold = 5
cv = StratifiedKFold(n_splits=fold, shuffle=True, random_state=42)
X_preds = np.zeros(st_train.shape[0])
preds = np.zeros(st_test.shape[0])
for i, (tr, ts) in enumerate(cv.split(st_train,Y)):
    print(ts.shape)
    mod = clf.fit(st_train[tr], Y[tr])
    X_preds[ts] = mod.predict(st_train[ts])
    preds += mod.predict(st_test)
    print("fold {}, ROC AUC: {:.3f}".format(i, r2_score(Y[ts], X_preds[ts])))
score = r2_score(Y, X_preds)
print(score)
preds1 = preds/fold

(1278,)
fold 0, ROC AUC: 0.988
(1275,)
fold 1, ROC AUC: 0.973
(1274,)
fold 2, ROC AUC: 0.987
(1274,)
fold 3, ROC AUC: 0.976
(1273,)
fold 4, ROC AUC: 0.975
0.9797408573122176


In [38]:
pred_lgb = preds1

In [40]:
pred_lgb_tune = preds1

In [43]:
preds1 = (pred_lgb + pred_lgb_tune)/2

In [33]:
set(preds1)

{1.0,
 1.2,
 1.4,
 1.8,
 2.0,
 2.2,
 2.6,
 3.0,
 4.0,
 4.2,
 4.6,
 5.0,
 5.8,
 6.0,
 6.2,
 7.0}

In [48]:
preds1 = [round(x) for x in preds1]

In [49]:
np.savetxt('submission_q2.csv',preds1)
##0.9759 baseline

38

In [19]:
%%time
from skopt import BayesSearchCV


def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest MSE: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")

bayes_cv_tuner = BayesSearchCV(
    estimator = lgb.LGBMClassifier( boosting_type='gbdt', subsample=0.6143), #colsample_bytree=0.6453, subsample=0.6143
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'num_leaves': (10, 100),      
        'max_depth': (0, 50),
        'min_child_samples': (0, 50),
        'max_bin': (100, 1000),
        'subsample_freq': (0, 10),
        'min_child_weight': (0, 10),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'scale_pos_weight': (1e-6, 500, 'log-uniform'),
        'n_estimators': (50, 150),
    },    
    scoring = 'r2', #neg_mean_squared_log_error
    cv = KFold(
        n_splits=5,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 1,
    n_iter = 100,   
    verbose = 0,
    refit = True,
    random_state = 42
)

# Fit the model
result = bayes_cv_tuner.fit(train2, Y, callback=status_print)

Model #1
Best MSE: 0.9271
Best params: {'learning_rate': 0.06610098295419149, 'max_bin': 755, 'max_depth': 47, 'min_child_samples': 16, 'min_child_weight': 7, 'n_estimators': 91, 'num_leaves': 42, 'reg_alpha': 0.004524161584138917, 'reg_lambda': 4.5035991909114364e-06, 'scale_pos_weight': 0.4316379249903662, 'subsample_freq': 5}

Model #2
Best MSE: 0.9394
Best params: {'learning_rate': 0.4729080547055919, 'max_bin': 895, 'max_depth': 15, 'min_child_samples': 48, 'min_child_weight': 9, 'n_estimators': 56, 'num_leaves': 22, 'reg_alpha': 1.521551227197179e-06, 'reg_lambda': 0.042535272805117035, 'scale_pos_weight': 196.9224481160877, 'subsample_freq': 6}

Model #3
Best MSE: 0.9394
Best params: {'learning_rate': 0.4729080547055919, 'max_bin': 895, 'max_depth': 15, 'min_child_samples': 48, 'min_child_weight': 9, 'n_estimators': 56, 'num_leaves': 22, 'reg_alpha': 1.521551227197179e-06, 'reg_lambda': 0.042535272805117035, 'scale_pos_weight': 196.9224481160877, 'subsample_freq': 6}

Model #4
B


Model #26
Best MSE: 0.9595
Best params: {'learning_rate': 0.22484328084594926, 'max_bin': 340, 'max_depth': 50, 'min_child_samples': 25, 'min_child_weight': 10, 'n_estimators': 150, 'num_leaves': 62, 'reg_alpha': 0.1538840221549436, 'reg_lambda': 0.00017394665539169078, 'scale_pos_weight': 8.020903509621036, 'subsample_freq': 0}

Model #27
Best MSE: 0.9595
Best params: {'learning_rate': 0.22484328084594926, 'max_bin': 340, 'max_depth': 50, 'min_child_samples': 25, 'min_child_weight': 10, 'n_estimators': 150, 'num_leaves': 62, 'reg_alpha': 0.1538840221549436, 'reg_lambda': 0.00017394665539169078, 'scale_pos_weight': 8.020903509621036, 'subsample_freq': 0}

Model #28
Best MSE: 0.9595
Best params: {'learning_rate': 0.22484328084594926, 'max_bin': 340, 'max_depth': 50, 'min_child_samples': 25, 'min_child_weight': 10, 'n_estimators': 150, 'num_leaves': 62, 'reg_alpha': 0.1538840221549436, 'reg_lambda': 0.00017394665539169078, 'scale_pos_weight': 8.020903509621036, 'subsample_freq': 0}

Mod


Model #52
Best MSE: 0.9786
Best params: {'learning_rate': 0.4435233092169438, 'max_bin': 612, 'max_depth': 0, 'min_child_samples': 50, 'min_child_weight': 3, 'n_estimators': 150, 'num_leaves': 10, 'reg_alpha': 1e-09, 'reg_lambda': 0.00027326438463215594, 'scale_pos_weight': 499.99999999999994, 'subsample_freq': 0}

Model #53
Best MSE: 0.9801
Best params: {'learning_rate': 0.3264540354731975, 'max_bin': 100, 'max_depth': 40, 'min_child_samples': 46, 'min_child_weight': 1, 'n_estimators': 150, 'num_leaves': 10, 'reg_alpha': 4.632845512742032e-08, 'reg_lambda': 0.00103612872552685, 'scale_pos_weight': 0.2740310347625757, 'subsample_freq': 0}

Model #54
Best MSE: 0.9801
Best params: {'learning_rate': 0.3264540354731975, 'max_bin': 100, 'max_depth': 40, 'min_child_samples': 46, 'min_child_weight': 1, 'n_estimators': 150, 'num_leaves': 10, 'reg_alpha': 4.632845512742032e-08, 'reg_lambda': 0.00103612872552685, 'scale_pos_weight': 0.2740310347625757, 'subsample_freq': 0}

Model #55
Best MSE: 


Model #78
Best MSE: 0.9868
Best params: {'learning_rate': 0.4725970210346864, 'max_bin': 100, 'max_depth': 31, 'min_child_samples': 42, 'min_child_weight': 0, 'n_estimators': 150, 'num_leaves': 10, 'reg_alpha': 1e-09, 'reg_lambda': 8.389301713582564e-05, 'scale_pos_weight': 499.99999999999994, 'subsample_freq': 0}

Model #79
Best MSE: 0.9868
Best params: {'learning_rate': 0.4725970210346864, 'max_bin': 100, 'max_depth': 31, 'min_child_samples': 42, 'min_child_weight': 0, 'n_estimators': 150, 'num_leaves': 10, 'reg_alpha': 1e-09, 'reg_lambda': 8.389301713582564e-05, 'scale_pos_weight': 499.99999999999994, 'subsample_freq': 0}

Model #80
Best MSE: 0.9868
Best params: {'learning_rate': 0.4725970210346864, 'max_bin': 100, 'max_depth': 31, 'min_child_samples': 42, 'min_child_weight': 0, 'n_estimators': 150, 'num_leaves': 10, 'reg_alpha': 1e-09, 'reg_lambda': 8.389301713582564e-05, 'scale_pos_weight': 499.99999999999994, 'subsample_freq': 0}

Model #81
Best MSE: 0.9868
Best params: {'learni